In [99]:
import requests
from bs4 import BeautifulSoup
import datetime
import pandas as pd

In [108]:
pagina_web = requests.get("https://www.cenace.gob.mx/graficademanda.aspx")
print(pagina_web) # 200 respuesta positiva

<Response [200]>


In [109]:
soup = BeautifulSoup(pagina_web.text, "html.parser")

In [110]:
datos = []
clave_region = ["ccbc", "ccbcsur", "cccentral", "ccnoreste", "ccnoroeste", 
                "ccnorte", "ccoccidental", "ccoriental", "ccpeninsular", "sinacional"]

In [111]:
region = "ccbcsur"
datos_region = soup.find("li", class_ = region)
datos_region

<li class="ccbcsur">
<span class="titlex">Baja California Sur</span>
<p>
                                Demanda Neta:
                                <span class="bold" id="ContentPlaceHolder1_demandaBCS">584 MW</span>
</p>
<p>
                                Generación Neta:
                                <span class="bold" id="ContentPlaceHolder1_generacionBCS">598 MW</span>
</p>
<p>
                                Pronóstico Neto:
                                <span class="bold" id="ContentPlaceHolder1_enlaceBCS">00 MW</span>
</p>
<p style="font-size: 11px; margin-top:10px">
<span class="bold" id="ContentPlaceHolder1_datetimeBCS">02/08/2023 12:30:45 a. m.</span>
</p>
</li>

In [112]:
for clave in clave_region:
    print(clave)

ccbc
ccbcsur
cccentral
ccnoreste
ccnoroeste
ccnorte
ccoccidental
ccoriental
ccpeninsular
sinacional


In [113]:
fecha = datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S")
for clave in clave_region:
        
      titulo_region = soup.find("li", class_ = clave)
      region = titulo_region.select_one(".titlex").text

      p_tags = soup.select("li." + clave + " p") # Selecciono las 3 etiquetas p
      for p_tag in p_tags:
            if "Demanda Neta:" in p_tag.text:
                 demanda_neta = float(p_tag.select_one(".bold").text.strip().replace(",","")[:-3]) 
            elif "Generación Neta" in p_tag.text:
                 generacion_neta = float(p_tag.select_one(".bold").text.strip().replace(",","")[:-3])
            elif "Pronóstico Neto" in p_tag.text:
                 pronostico_neto = p_tag.select_one(".bold").text.strip().replace(",","")[:-3]     
               
      datos.append({
              "Fecha" : fecha,
              "Región": region,
              "Generación Neta MW" : generacion_neta,
              "Demanda Neta MW" : demanda_neta,
              "Diferencia MW" : generacion_neta - demanda_neta,
              "Pronóstico Neto MW" : pronostico_neto
            })
      
df = pd.DataFrame(datos)
df

,Fecha,Región,Generación Neta MW,Demanda Neta MW,Diferencia MW,Pronóstico Neto MW
0,02-08-2023 00:32:46,Baja California,2475.0,2446.0,29.0,00
1,02-08-2023 00:32:46,Baja California Sur,598.0,584.0,14.0,00
2,02-08-2023 00:32:46,CENTRAL,3912.0,6016.0,-2104.0,00
3,02-08-2023 00:32:46,NORESTE,12426.0,9988.0,2438.0,00
4,02-08-2023 00:32:46,NOROESTE,4771.0,4785.0,-14.0,00
5,02-08-2023 00:32:46,NORTE,4126.0,4425.0,-299.0,00
6,02-08-2023 00:32:46,OCCIDENTAL,6193.0,8865.0,-2672.0,00
7,02-08-2023 00:32:46,ORIENTAL,10360.0,6885.0,3475.0,00
8,02-08-2023 00:32:46,PENINSULAR,1334.0,2250.0,-916.0,00
9,02-08-2023 00:32:46,S.I. NACIONAL,43128.0,43211.0,-83.0,43377
